In [ ]:
from keras.models import save_model,load_model
from numpy import save,load
from pymatgen.core import Structure, Lattice
from pymatgen.core import Molecule
from pymatgen.analysis.adsorption import *
from pymatgen.core.surface import generate_all_slabs
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.ext.matproj import MPRester
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.io.vasp.inputs import Kpoints
from pymatgen.io.vasp.inputs import Incar
from pymatgen.io.vasp.inputs import Potcar
from pymatgen.io.vasp.inputs import PotcarSingle
from pymatgen.io.vasp.sets import MPRelaxSet
from pymatgen.ext.matproj import MPRester
from pymatgen.io.ase import AseAtomsAdaptor
import random
import tensorflow as tf
from dscribe.descriptors import SOAP
from dscribe.descriptors import MBTR,ACSF,CoulombMatrix,EwaldSumMatrix,SineMatrix 
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
import numpy as np
from sko.GA import GA
import time
import datetime
from sko.tools import set_run_mode
my_key='BJU2qDd4gpCPO0rf'
m= MPRester(my_key)
slab_basic=Structure.from_file('IrO2_110_init.cif')
seed=1
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
#####decribe parameters#####
rcut = 6
#####decribe parameters#####
sm = SineMatrix(
    n_atoms_max=120,
    permutation="sorted_l2",
    sparse=False,
    flatten=False)
ew=EwaldSumMatrix(
    n_atoms_max=120,
    permutation="none",flatten=False,sparse=False)
acsf=ACSF(species=['Eu','Sc','V','Cu','Ir','Pt','Au','O'],
    rcut=rcut,
    periodic=True,
    g2_params=[[1, 1], [1, 2], [1, 3]],
    g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]])

In [ ]:

def generate_costly_function(task_type='io_costly'):
    # generate a high cost function to test all the modes
    # cost_type can be 'io_costly' or 'cpu_costly'
    if task_type == 'io_costly':
        def costly_function():
            time.sleep(0.1)
            return 1
    else:
        def costly_function():
            n = 10000
            step1 = [np.log(i + 1) for i in range(n)]
            step2 = [np.power(i, 1.1) for i in range(n)]
            step3 = sum(step1) + sum(step2)
            return step3

    return costly_function
def substitue_slab(ele_list,sub_list,slab):
    slab_copied=slab.copy()
    for i in range(0,len(ele_list)):
        ele_each=ele_list[i]
        sub_each=sub_list[i]
        slab_copied[sub_each]=ele_each
    return slab_copied
def mkdir(path): 
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        print ("---  new folder...  ---")
        print ("---  OK  ---") 
    else:
        print ("---  There is this folder!  ---")
def int_list(target_list):
    new_list=[]
    for i in range(0,len(target_list)):
        num=int(target_list[i])
        new_list.append(num)
    return new_list
def generate_des(Read_Struct_SLAB,describer1,describer2,describer3):
    des1=describer1.create(Read_Struct_SLAB,n_jobs=-1)
    des2=describer2.create(Read_Struct_SLAB,n_jobs=-1)
    des3=describer3.create(Read_Struct_SLAB,n_jobs=-1)
    new_des=np.concatenate((des1,des2),axis=1)
    new_des=np.concatenate((new_des,des3),axis=1)
    new_des=new_des.reshape(240,208)
    return new_des    
def predict_SLAB_energy(model,replaced_slab):
    ase_slab=AseAtomsAdaptor.get_atoms(replaced_slab)
    des=generate_des(ase_slab,sm,ew,acsf)
    transed_input=Minmaxsc.fit_transform(des)
    transed_input=transed_input.reshape(-1, transed_input.shape[0], transed_input.shape[1], 1)
    raw_output=model.predict(transed_input[0:1])
    transed_output=Rsc2.inverse_transform(raw_output)
    print(transed_output[0][0])
    return transed_output[0][0]
def GA_predictor(p):
    start_time = datetime.datetime.now()
    ele_1_serial,ele_2_serial,ele_3_serial,ele_4_serial,ele_5_serial,ele_6_serial=p
    a=[ele_1_serial,ele_2_serial,ele_3_serial,ele_4_serial,ele_5_serial,ele_6_serial]
    b=list(set(a))
    if a==b:
        replaced_serial_list=[ele_1_serial,ele_2_serial,ele_3_serial,ele_4_serial,ele_5_serial,ele_6_serial]
        replaced_serial_list=int_list(replaced_serial_list)
        replaced_slab=substitue_slab(ele_list,replaced_serial_list,slab_basic)
        current_slab_energy=predict_SLAB_energy(model,replaced_slab)
    else:
        current_slab_energy=1000
    time_costs=(datetime.datetime.now() - start_time)
    print('time cost per step',time_costs, 's')
    return current_slab_energy
######specify the input and output######
data_input=np.load('data_input_sm_ew_acsf_rcu6.npy')
data_output=np.load('data_output_sm_ew_acsf_rcu6_SLAB.npy')
####scaler construction####
Minmaxsc  = MinMaxScaler(feature_range=(0, 1))
Minmaxsc2  = MinMaxScaler(feature_range=(0, 1))
Stdsc  = StandardScaler()
Stdsc2  = StandardScaler()
MAsc  = MaxAbsScaler()
MAsc2  = MaxAbsScaler()
Rsc  = RobustScaler()
Rsc2  = RobustScaler()

X = Minmaxsc.fit_transform(data_input.reshape(-1, data_input.shape[-1])).reshape(data_input.shape)
y = Rsc2.fit_transform(data_output.reshape(-1,1))
random_seed=1580
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=random_seed,test_size=0.1)
X_train = X_train.reshape(-1, X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(-1, X_train.shape[1], X_train.shape[2], 1)

In [ ]:
def serial_independent(x):
    return (x[0]!=x[1]!=x[2]!=x[3]!=x[4]!=x[5])-1
constraint_eq = [
    serial_independent
    ]


In [ ]:
###############
ele_list=['Sc','V','Eu','Cu','Pt','Au']
model=load_model('CNN.h5')

In [ ]:
task_type='cpu_costly'
costly_function = generate_costly_function(task_type=task_type)
# to use the vectorization mode, the function itself should support the mode.
mode = 'common'
# mode ='vectorization'
set_run_mode(GA_predictor, mode)
ga = GA(func=GA_predictor, n_dim=6, size_pop=1000, max_iter=500,prob_mut=0.15,lb=[0,0,0,0,0,0], ub=[39,39,39,39,39,39],
        constraint_eq=constraint_eq, precision=[1,1,1,1,1,1])
start_time = datetime.datetime.now()
best_x, best_y = ga.run()
print('best_x:', best_x, '\n', 'best_y:', best_y)
print('on {task_type} task,use {mode} mode, costs {time_costs}s'
      .format(task_type=task_type, mode=mode,
              time_costs=(datetime.datetime.now() - start_time).total_seconds()))

In [ ]:
task_type='cpu_costly'
costly_function = generate_costly_function(task_type=task_type)
# to use the vectorization mode, the function itself should support the mode.
mode = 'common'
# mode ='vectorization'
set_run_mode(GA_predictor, mode)
ga = GA(func=GA_predictor, n_dim=6, size_pop=1000, max_iter=1000,prob_mut=0.15,lb=[0,0,0,0,0,0], ub=[39,39,39,39,39,39],
        constraint_eq=constraint_eq, precision=[1,1,1,1,1,1])
start_time = datetime.datetime.now()
best_x, best_y = ga.run()
print('best_x:', best_x, '\n', 'best_y:', best_y)
print('on {task_type} task,use {mode} mode, costs {time_costs}s'
      .format(task_type=task_type, mode=mode,
              time_costs=(datetime.datetime.now() - start_time).total_seconds()))